## Loading data

In [2]:
from datasets import load_dataset

train_dataset = load_dataset("xlangai/spider", split='train')
validation_dataset = load_dataset("xlangai/spider", split='validation')
db_schema = load_dataset("richardr1126/spider-schema", split='train')

In [2]:
print(train_dataset)
print(validation_dataset)

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 7000
})
Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 1034
})


In [3]:
print(db_schema)

Dataset({
    features: ['db_id', 'Schema (values (type))', 'Primary Keys', 'Foreign Keys'],
    num_rows: 166
})


In [4]:
train_dataset[0]

{'db_id': 'department_management',
 'query': 'SELECT count(*) FROM head WHERE age  >  56',
 'question': 'How many heads of the departments are older than 56 ?',
 'query_toks': ['SELECT',
  'count',
  '(',
  '*',
  ')',
  'FROM',
  'head',
  'WHERE',
  'age',
  '>',
  '56'],
 'query_toks_no_value': ['select',
  'count',
  '(',
  '*',
  ')',
  'from',
  'head',
  'where',
  'age',
  '>',
  'value'],
 'question_toks': ['How',
  'many',
  'heads',
  'of',
  'the',
  'departments',
  'are',
  'older',
  'than',
  '56',
  '?']}

In [5]:
filtered_data = db_schema.filter(lambda row: row['db_id'] == "department_management")
print(filtered_data[0])


{'db_id': 'department_management', 'Schema (values (type))': 'department : Department_ID (number) , Name (text) , Creation (text) , Ranking (number) , Budget_in_Billions (number) , Num_Employees (number) | head : head_ID (number) , name (text) , born_state (text) , age (number) | management : department_ID (number) , head_ID (number) , temporary_acting (text)', 'Primary Keys': 'department : Department_ID | head : head_ID | management : department_ID', 'Foreign Keys': 'management : head_ID equals head : head_ID | management : department_ID equals department : Department_ID'}


## Use llama for generation (0-shot)

In [3]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
!pip -qq install langchain
!pip -qq install langchain-core
!pip -qq install langchain-community

In [9]:
from langchain_community.llms import Ollama
from llm import LLM

llm = LLM(model=Ollama(model="llama3.2"))
llm.model.invoke("hello")

/var/folders/h1/czlwlk_50vq1xdyv651xbn280000gn/T/ipykernel_53370/2581367007.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = LLM(model=Ollama(model="llama3.2"))


"Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?"

### Trying to generate some SQL queries (0-shot)

In [11]:
nb_queries = 5
prompt = "Write the SQL query that answer the user's question. Answer only the SQL query, write SQL operators (COUNT, AVG, etc.) in MAJ. Question: {question}.\nSQL Query:"
classification_prompt = "Tell if these two SQL queries are giving the same result, answer yes or no only. If no, explain. Query 1: {query1}.\nQuery 2: {query2}.\nSame (correction if necessary):"

for i in range(nb_queries):
    dataset_i = train_dataset[i]
    question = dataset_i["question"]
    print(f"\n--------\n")
    print(f"question: {question}")
    query1 = dataset_i['query']
    prompt_completed = prompt.format(question=question)
    query2 = llm.model.invoke(prompt_completed)
    print(f"\nAnswer: {query2}\n")
    print(f"Correct answer: {dataset_i['query']}\n")

    correct = llm.model.invoke(classification_prompt.format(query1 = query1, query2 = query2))
    print(f"Correct: {correct}")
    



--------

question: How many heads of the departments are older than 56 ?

Answer: SELECT COUNT(*) FROM Employees WHERE Age > 56 AND Department = 'Head'

Correct answer: SELECT count(*) FROM head WHERE age  >  56

Correct: No.

The difference lies in the table names and the conditions applied to those tables. Query 1 is looking at a "head" table, whereas Query 2 is referencing an "Employees" table with a condition on the "Department" column being equal to 'Head'. They are likely referencing different datasets or tables in your database.

--------

question: List the name, born state and age of the heads of departments ordered by age.

Answer: SELECT e1.name, e2.state, TIMESTAMPDIFF(YEAR, e2.born, CURDATE()) AS age FROM employees e1 JOIN employees e2 ON e1.manager = e2.emp_no GROUP BY e1.name, e2.state ORDER BY e2.born;

Correct answer: SELECT name ,  born_state ,  age FROM head ORDER BY age

Correct: No.

Query 1 is a simple select with an order by clause. It will return all rows in t

### Compute accuracy

In [12]:
from parameters import classification_prompt, prompt_schema

def generate_query(question, llm, prompt=prompt_schema, schema=None):
    """
    """
    prompt_completed = prompt.format(question=question, schema=schema)
    generated_query = llm.model.invoke(prompt_completed)

    return generated_query

In [10]:
import re

score_prompt = """Determine the degree of logical equivalence between the two SQL queries, assuming the same schema, data, and execution environment. Provide a score between 0 and 1, where:

- 1: Fully logically equivalent (queries produce identical results under all circumstances).
- 0: Completely different (queries are logically unrelated or produce entirely different results).
- Scores between 0 and 1 should reflect partial equivalence, considering factors such as:
  - Differences in filters, conditions, or joins that partially overlap.
  - Minor variations in selected columns or formatting that do not affect the overall logic.
  - Similar intent but differing specifics in query structure.

Explain your score briefly, highlighting key differences or similarities that influenced the rating.

Query:
{query}

Generated query:
{generated_query}

Equivalence Score (0-1, with explanation):"""

def equivalence_score(generated_query, query, llm, score_prompt=score_prompt) -> bool:
    """
    Return an equivalence score between generated query and a groundtruth query with an orchestrator LLM
    """
    pattern = r"\s*([0-9]*\.?[0-9]+)"
    explanation = llm.invoke(score_prompt.format(query = query, generated_query = generated_query))
    match = re.search(pattern, explanation, flags=re.IGNORECASE)

    score = 0

    if match:
        score = float(match.group(1))
    
    return score, explanation


In [16]:
def is_correct(generated_query, query, llm, classification_prompt=classification_prompt) -> bool:
    """
    Return true if generated query is considered as equivalent to query in terms of result by an orchestrator LLM
    """
    pattern = r'\b(yes|no)\b'
    correct = llm.invoke(classification_prompt.format(query = query, generated_query = generated_query))
    matches = re.findall(pattern, correct, flags=re.IGNORECASE)

    return "Yes" in matches or "yes" in matches


In [21]:
import re
from tqdm import tqdm
from parameters import classification_prompt, prompt_schema

# pattern = r'\b(yes|no)\b'
nb_queries = 10

verbose = False
nb_correct = 0
list_incorrect = []

for i in tqdm(range(nb_queries), desc="Processing queries"):
    dataset_i = train_dataset[i]
    db_id = dataset_i['db_id']
    filtered_data = db_schema.filter(lambda row: row['db_id'] == db_id)
    schema = filtered_data[0]
    
    query = dataset_i['query']
    question = dataset_i['question']
    
    generated_query = llm.generate_query(question, prompt_schema, schema)
    correct = llm.is_correct(generated_query, query, classification_prompt)

    # Increment nb_yes for each "yes" found
    if correct:
        nb_correct += 1

    if not correct:
        list_incorrect.append({
            'llm_answer':generated_query,
            'correct_answer':query,
            'classification':correct
        })

    if verbose:
        print(f"\n--------")
        print(f"question: {question}")
        print(f"schema: {schema}")
        print(f"\nAnswer: {generated_query}\n")
        print(f"Correct answer: {dataset_i['query']}")
        print(f"Correct: {correct}")

print(f"Accuracy: {nb_correct/nb_queries}")


Processing queries: 100%|██████████| 10/10 [00:31<00:00,  3.13s/it]

Accuracy: 0.2


In [18]:
for i in range(len(list_incorrect)):
    print("\n")
    print(f"LLM answer: {list_incorrect[i]['llm_answer']}")
    print(f"Correct answer: {list_incorrect[i]['correct_answer']}")
    print(f"Correct: {list_incorrect[i]['classification']}")



LLM answer: SELECT MAX(Budget_in_Billions) ,  MIN(Budget_in_Billions) FROM department
Correct answer: SELECT max(budget_in_billions) ,  min(budget_in_billions) FROM department
Correct: False


LLM answer: SELECT avg(Num_Employees) FROM department WHERE Ranking BETWEEN 10 AND 15;
Correct answer: SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15
Correct: False


In [ ]:
train_dataset[0]

### utils functions

In [5]:
%pip install -qq -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from utils import compute_metrics, generate_synthetic_db
from parameters import classification_prompt

test_data = train_dataset[0]

db_id = test_data['db_id']
question = test_data['question']
query = test_data['query']
generated_query = llm.generate_query(question)

print(f"Question: {question}")
print(f"Query: {query}")
print(f"Generate_query: {generated_query}")

correct = llm.is_correct(generated_query, query, classification_prompt)
equivalence_score, explanation = llm.equivalence_score(generated_query, query)


# Generate synthetic database
# TODO
valid_pred, keyword_score, identifier_score = compute_metrics(generated_query, query, test_data['db_id'], synth_db)

Question: How many heads of the departments are older than 56 ?
Query: SELECT count(*) FROM head WHERE age  >  56
Generate_query: SELECT COUNT(*) FROM departments WHERE manager_id IN ( SELECT id FROM employees WHERE age > 56 );
<class 'dict'>
<class 'datasets.arrow_dataset.Dataset'>


AttributeError: 'dict' object has no attribute 'iterrows'

In [7]:
print(f"Correct: {correct}")
print(f"valid_pred: {valid_pred}")
print(f"keyword_score: {keyword_score}")
print(f"identifier_score: {identifier_score}")
print(f"Equivalence score: {equivalence_score}")
print(f"Explanation: {explanation}")

Correct: False
valid_pred: True
keyword_score: 1.0
identifier_score: 0.0
Equivalence score: 0.5
Explanation: I would assign an equivalence score of 0.5 to these two SQL queries.

The primary reason for this score is that the generated query (Query 2) has a different table name (`department_heads`) compared to Query 1 (`head`). This difference in table name means that Query 2 will return different data than Query 1, as it only considers rows in `department_heads` with an age greater than 56.

However, there are minor similarities between the two queries. Both use the same filter condition (`age > 56`) and both select the count of rows that satisfy this condition. Additionally, the queries have a similar structure, using the `SELECT COUNT(*)` syntax to count the number of rows meeting the specified criteria.

Overall, while Query 2 is not fully logically equivalent to Query 1 due to the difference in table name, it shares some similarities in its logic and structure, which warrants a sco

# Metrics

In [31]:
import pandas as pd
ds = load_dataset("xlangai/spider")
db_schema = pd.DataFrame(load_dataset('richardr1126/spider-schema', split='train'))

In [ ]:
# Generate synthetix data
try:
    sql_databases = generate_synthetic_db(db_schema)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Compute metrics on synth data

import itertools

k = 20
print(f"\n--------------\n")
nb_err = 0
for data in itertools.islice(ds['train'], k):
    question = data['question']
    query1 = data['query']
    db_id = data['db_id']
    schema = db_schema[db_schema['db_id'] == db_id]

    generated_query = llm.generate_query(question, schema=schema)
    metrics = compute_metrics(query1, generated_query, db_id, sql_databases)
    errs = dict()
    if metrics['result_metric'] == 0:
        nb_err += 1
        errs['query'] = query1
        errs['gen_query'] = generated_query
    print(f"\nError for query:")
    print(f"query: {query1}")
    print(f"gen query: {generated_query}")
    print(f"metrics:")
    print(metrics)

print(f"\n\nProportion of errs")
print(nb_err/k)


>>> ERROR - i=95 / db : world_1 / table : sqlite_sequence
object name reserved for internal use: sqlite_sequence 

CREATE TABLE "sqlite_sequence" (
    "name" TEXT,
    "seq" TEXT
)
INSERT INTO "sqlite_sequence" ("name", "seq") VALUES (?, ?) 

>>> ERROR - i=111 / db : soccer_1 / table : sqlite_sequence
object name reserved for internal use: sqlite_sequence 

CREATE TABLE "sqlite_sequence" (
    "name" TEXT,
    "seq" TEXT
)
INSERT INTO "sqlite_sequence" ("name", "seq") VALUES (?, ?) 

>>> ERROR - i=128 / db : store_1 / table : sqlite_sequence
object name reserved for internal use: sqlite_sequence 

CREATE TABLE "sqlite_sequence" (
    "name" TEXT,
    "seq" TEXT
)
INSERT INTO "sqlite_sequence" ("name", "seq") VALUES (?, ?) 


--------------

ERROR (db :department_management) : no such column: age

Error for query:
query: SELECT count(*) FROM head WHERE age  >  56
gen query: SELECT COUNT(*) FROM management WHERE age > 56
metrics:
{'valid_pred': True, 'keyword_score': 1.0, 'identifiers_s

## Retrieval

We will implement a retrieval tool for generating the SQL Query. The goal is to retrieve relevant data: schemas of relevant databases
Indeed, for the moment we put some database schemas in the prompt context, however this may be illegal as being information leak. In a real case production text2SQL tool, we may not have a table schema linked to a user query. Moreover, a query may need to join table, to retrieve K database schema may be efficient

In [ ]:
# Lets retrieve relevant schemas

